## Libraries

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
from tqdm import tqdm_notebook
import time
import os

from web_scrapping_functions import *

# Web Scrapping

In [ ]:
url = 'https://www.drinklab.org/cocktail-recipes/'
html = requests.get(url).content

In [ ]:
soup = BeautifulSoup(html, "html.parser") 

### Extract urls to 'Cocktails by Letter'

In [ ]:
cocktails_by_letter = soup.find_all('div', class_="x-text e89516-4 m1x2k-3")

url_by_letter = []

for link in cocktails_by_letter:
    anchors = link.find_all('a')
    for anchor in anchors:
        url = anchor['href']
        url_by_letter.append(url)


url_by_letter

### Iterate in the url_by_letter list and get html soup from each from which to extract the url to each individual cocktail page.

In [ ]:
cocktail_urls = []

for url in url_by_letter:
    base_url = url
    start_page = 1

    while True:
        url = base_url + "?_page=" + str(start_page)
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        divs = soup.find_all('div', class_="col-md-3 col-sm-4 col-xs-6 pt-cv-content-item pt-cv-1-col")
        
        
        temp_urls = []

        for div in divs:
            anchor = div.find('a')
            if anchor is not None and 'href' in anchor.attrs:
                temp_urls.append(anchor["href"])

        cocktail_urls.extend(temp_urls)

        pagination = soup.find('ul', class_="pt-cv-pagination")
        if pagination is None or 'data-totalpages' not in pagination.attrs:
            # print('Next subpage')
            break
        
        total_pages = int(pagination['data-totalpages'])
        if start_page >= total_pages:
            # print('Next page')
            break
        
        start_page += 1

    # print(cocktail_urls)


In [ ]:
len(cocktail_urls)

In [ ]:
cocktail_urls

## Extract relevant info for each cocktail from cocktail_urls list

Create lists to receive scrapped material, to be concatenated into a Dataframe

In [ ]:
ctl_names = []
ctl_description = [] 
ctl_preptime = []
ctl_ingredients = []
ctl_recipe = []
ctl_vid = []
ctl_nutrition = []

une mega loop qui récup_re toute les soups, les write en html.
et ensuite on va chercher dans le soups tous les trucs que je veux (name, description, etc...)
+
add timer around get

Scrap and stock locally all HTML soups, for futur iterating.

In [ ]:
test_urls = ['https://www.drinklab.org/american-beauty-cocktail/',
 'https://www.drinklab.org/aviation-gin-riblet/',
 'https://www.drinklab.org/mason-jar-aperol-spritz/',
 'https://www.drinklab.org/americano/',
 'https://www.drinklab.org/amaretto-sour-2/',
 'https://www.drinklab.org/aperol-spritz/',
 'https://www.drinklab.org/airedale-cocktail/',
 'https://www.drinklab.org/articuno-pokemon-cocktail/',
 'https://www.drinklab.org/alfonso-2/',
 'https://www.drinklab.org/azzuro-2/',
 'https://www.drinklab.org/american-leroy-2/',
 'https://www.drinklab.org/angels-tit/',
 'https://www.drinklab.org/alex-chi-chi-2/',
 'https://www.drinklab.org/aqua-2/',
 'https://www.drinklab.org/apple-colada-2/',
 'https://www.drinklab.org/arago-2/',
 'https://www.drinklab.org/arizona-twister-2/',
 'https://www.drinklab.org/andreas-colada-collision-2/',
 'https://www.drinklab.org/almond-cooler-2/',
 'https://www.drinklab.org/almond-joy-2-3/',
 'https://www.drinklab.org/angels-tip-2-2/',
 'https://www.drinklab.org/american-glory-2/',
 'https://www.drinklab.org/american-rose-2/',
 'https://www.drinklab.org/american-flyer-2/',
 'https://www.drinklab.org/aqua-marina-2/' ]

In [21]:
def get_soups(url_list):

    cocktail_htmls = []
    file_counter = 622
    file_path = 'C:/Users/User/Desktop/FORMATION/IRONHACK/PROJECTS/PROJECT-FINAL/html_soup'

    for url in tqdm_notebook(url_list):

        response = requests.get(url)

        soup = BeautifulSoup(response.content, 'lxml')

        cocktail_htmls.append(soup)

        filename = f'cocktail_{file_counter}.html'

        file_full_path = os.path.join(file_path, filename)
        
        with open(file_full_path, 'w', encoding='utf-8') as file:
            file.write(str(soup))

        file_counter += 1

In [22]:
get_soups(cocktail_urls[622:])

C:\Users\User\AppData\Local\Temp\ipykernel_14140\3470693433.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for url in tqdm_notebook(url_list):


  0%|          | 0/4505 [00:00<?, ?it/s]

___________________________

In [ ]:
file_path = 'C:/Users/User/Desktop/FORMATION/IRONHACK/PROJECTS/PROJECT-FINAL/html_soup'

# extracted_info = []

In [ ]:
def scrap_info(x):

    extracted_info = []

    for file_name in os.listdir(x):
        if file_name.endswith('.html'):
            file_full_path = os.path.join(x, file_name)
            
            with open(file_full_path, 'r', encoding='utf-8') as file:
                soup = BeautifulSoup(file, 'html.parser')
                
                # name scrap
                name = soup.find('h1', class_="wprm-recipe-name wprm-block-text-bold")
                if name is not None:
                    ctl_name = name.text
                else:
                    ctl_name = None
                if ctl_name is None:
                    continue


                # description scrap
                description = soup.find('div', class_="wprm-recipe-summary wprm-block-text-normal")

                if description is not None:
                    description_txt = soup.find('div', class_="wprm-recipe-summary wprm-block-text-normal").text
                    ctl_description.append(description_txt)

                else:
                    ctl_description.append("Unfortunately, we have no description for this drink... You'll have to describe it yourself!")
                

                # recipe scrap
                recipe = soup.find('div', class_="wprm-recipe-instruction-group")
                if recipe is not None:
                    ctl_recipe = recipe.text
                else:
                    ctl_recipe = "Woops... We couldn't retrieve the exact recipe... It's trial & error time! Just a little more fun before enjoying a nice drink!"
                

                # ingredients scrap
                ingredients = soup.find('div', class_="wprm-recipe-ingredient-group")
                if ingredients is not None:
                    ctl_ingredients = ingredients.text.replace('▢', ' |').replace('  ', ':')
                else:
                    ctl_ingredients = "Woops... What happened ?! Something didn't work, please try again."    
                # ingredients = [tag.text.strip() for tag in ingredient_tags]


                # video link scrap
                video = soup.find('iframe')
                if video is not None:
                    ctl_vid = video['src']
                else:
                    ctl_vid = "There doesn't seem to be an instructional video for this cocktail. Why not make the tutorial yourself!"

                
                # append all scrapped info to list
                extracted_info.append([ctl_name, ctl_description, ctl_recipe, ctl_ingredients, ctl_vid])

    # Create dataframe from extracted_info list.
    df = pd.DataFrame(extracted_info, columns=['Name', 'Description', 'Recipe', 'Ingredients', 'Video Link'])

    # Print dataframe
    return df

In [ ]:
ctl_df = scrap_info(file_path)

In [ ]:
ctl_df

__________________________

Scrap cocktail names

In [ ]:
def get_names(url_list) :
      
      for url in tqdm_notebook(url_list):
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        names = soup.find('h1', class_="wprm-recipe-name wprm-block-text-bold")
      if names is None :
         print('Could not scrap ctl_name for {url}')
      else:
         ctl_names.append(names.text)
      

In [ ]:
get_names(cocktail_urls)

Scrap description

In [ ]:
def get_description(url_list):

   for url in url_list:
      response = requests.get(url)
      soup = BeautifulSoup(response.content, 'html.parser')
      description = soup.find('div', class_="wprm-recipe-summary wprm-block-text-normal")

      if description is not None:
         description_txt = soup.find('div', class_="wprm-recipe-summary wprm-block-text-normal").text
         ctl_description.append(description_txt)

      else:
         ctl_description.append("Unfortunately, we have no description for this drink... You'll have to describe it yourself!")

In [ ]:
get_description(cocktail_urls)

Scrap prep time

In [ ]:
def get_prep_time(url_list):

   for url in url_list:
      response = requests.get(url)
      soup = BeautifulSoup(response.content, 'html.parser')
      prep_time = soup.find('div', class_="wprm-recipe-block-container wprm-recipe-block-container-columns wprm-block-text-normal wprm-recipe-time-container wprm-recipe-total-time-container")

      if prep_time is not None:
         prep_time_txt = prep_time.text.replace(' minutes minutes',' minutes')
         ctl_preptime.append(prep_time_txt)

      else:
         ctl_preptime.append("No time estimate for the preparation... Only one way to find out!")

In [ ]:
get_prep_time(cocktail_urls)

Scrap ingredients

In [ ]:
def get_ingredients(url_list):

   for url in url_list:
      response = requests.get(url)
      soup = BeautifulSoup(response.content, 'html.parser')
      # ingredients = soup.find('div' , class_="wprm-recipe-ingredients-container wprm-recipe-ingredients-no-images wprm-recipe-87010-ingredients-container wprm-block-text-normal wprm-ingredient-style-regular wprm-recipe-images-before")
      ingredients = soup.find('div', class_="wprm-recipe-ingredient-group")

      if ingredients is not None:
         ingredients_txt = ingredients.text.replace('▢',' |').replace('  ',':')

         ctl_ingredients.append(ingredients_txt)

      else:
         ctl_ingredients.append("Woops... What happened ?! Something didn't work, please try again.")    

In [ ]:
get_ingredients(cocktail_urls)

Scrap recipe

In [ ]:
def get_recipe(url_list):

   for url in url_list:
      response = requests.get(url)
      soup = BeautifulSoup(response.content, 'html.parser')
      recipe = soup.find('div', class_="wprm-recipe-instruction-group")

      if recipe is not None:
         recipe_txt = recipe.text

         ctl_recipe.append(recipe_txt)

      else:
         ctl_recipe.append("Woops... We couldn't retrieve the exact recipe... It's trial & error time! Just a little more fun before enjoying a nice drink!") 

In [ ]:
get_recipe(cocktail_urls)

Scrap video

In [ ]:
def get_video(url_list):

   for url in url_list:
      response = requests.get(url)
      soup = BeautifulSoup(response.content, 'html.parser')
      video = soup.find('iframe')

      if video is not None:
         video_url = video['src']
         ctl_vid.append(video_url)

      else:
         ctl_vid.append("There doesn't seem to be an instructional video for this cocktail. Why not make the tutorial yourself!") 

In [ ]:
get_video(cocktail_urls)

Scrap nutrition facts

In [ ]:
def get_nutrition(url_list):

   for url in url_list:
      response = requests.get(url)
      soup = BeautifulSoup(response.content, 'html.parser')
      nutrition = soup.find('div', class_="wprm-nutrition-label-container wprm-nutrition-label-container-simple wprm-block-text-normal")

      if nutrition is not None:
         nutrition_txt = nutrition.text
         ctl_nutrition.append(nutrition_txt)

      else:
         ctl_nutrition.append("There doesn't seem to be any nutritional facts associated to this cocktail. But we know that's not why you're here...") 

In [ ]:
get_nutrition(cocktail_urls)

Check if scrap successful

In [ ]:
if len(ctl_names) == len(cocktail_urls):
     print('Ctl Names = OK!')
     else print('Ctl Names = TO CHECK!')

if len(ctl_description) == len(cocktail_urls):
     print('Ctl Names = OK!')
     else print('Ctl Names = TO CHECK!')

if len(ctl_preptime) == len(cocktail_urls):
     print('Ctl Names = OK!')
     else print('Ctl Names = TO CHECK!')

if len(ctl_ingredients) == len(cocktail_urls):
     print('Ctl Names = OK!')
     else print('Ctl Names = TO CHECK!')

if len(ctl_recipe) == len(cocktail_urls):
     print('Ctl Names = OK!')
     else print('Ctl Names = TO CHECK!')

if len(ctl_vid) == len(cocktail_urls):
     print('Ctl Names = OK!')
     else print('Ctl Names = TO CHECK!')

if len(ctl_nutrition) == len(cocktail_urls):
     print('Ctl Names = OK!')
     else print('Ctl Names = TO CHECK!')

_________________________________

### Debug the webscrapping function

return the names in the functions and deal with the append here.  
OR  
paass the list in the function and return the filled list

In [ ]:
# def webscrapping(x):
#     get_names(x)
#     get_description(x)
#     get_prep_time(x)
#     get_ingredients(x)
#     get_recipe(x)
#     get_video(x)
#     get_nutrition(x)

#     ctl_df = pd.DataFrame(list(zip(ctl_names,ctl_description,ctl_ingredients,ctl_recipe,ctl_preptime,ctl_vid,ctl_nutrition)))
#     ctl_df = ctl_df.set_axis(["Cocktail","Description","Ingredients","Recipe","Prep. Time","Video Link","Nutritional Facts"], axis="columns")
#     return ctl_df

In [ ]:
webscrapping(test_urls)

In [ ]:
ctl_names

_____________________________